In [9]:
 import sys
#Second append the folder path

sys.path.insert(0, './../inc')
sys.path.insert(0, './../src')

In [14]:
import os
from glob import glob
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from pyntcloud import PyntCloud
from pyntcloud import scalar_fields
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neighbors.kde import KernelDensity
from sklearn import svm, datasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import pickle

#import pclpy
# install this pip sudo apt install libgl1-mesa-glx
import open3d as o3d

import json
import pdal

#from pointcloud import cloud_features, cloud_to_featurs_datasets

#import keras
#import tensorflow

import pdal_pipline

In [2]:
classe_labels = {
    "noise/other"  : 7,
    "ground"       : 2,
    "road"         : 11,
    "vegetation"   : 3,
    "building"     : 6,
    "light"        : 19,
    "signal"       : 20,
    "vehicle"      : 21,
    "fence/people" : 22
    }

classe_labels

{'noise/other': 7,
 'ground': 2,
 'road': 11,
 'vegetation': 3,
 'building': 6,
 'light': 19,
 'signal': 20,
 'vehicle': 21,
 'fence/people': 22}

In [5]:
from pathlib import Path
extension = '.bpf'
files=[]
PrjPath = Path('./../../')
i = 0
for entry in PrjPath.rglob(f'*{extension}'): # PrjPath.iterdir():
    print(i, entry)
    files.append(entry)
    i+=1

#print(files)
#LIDAR_FILENAME = files[16].resolve()
#print(LIDAR_FILENAME)

extension = '.shp'
shp_files=[]

i = 0
for entry in PrjPath.rglob(f'*{extension}'): # PrjPath.iterdir():
    print(i, entry)
    shp_files.append(entry)
    i+=1
    
#print(files)
#LIDAR_FILENAME = files[16].resolve()
#print(LIDAR_FILENAME)

extension = '.las'
LAS_files=[]

i = 0
for entry in PrjPath.rglob(f'*{extension}'): # PrjPath.iterdir():
    print(i, entry)
    LAS_files.append(entry)
    i+=1

0 ../../Werk2_part1_features.bpf
1 ../../Werk2_part2_features.bpf
0 ../../Werk2_part1_features.shp
1 ../../shape/086b_classified.shp
0 ../../Werk2_part1_features-Copy2.las
1 ../../Werk2_part1_features.las
2 ../../Werk2_part1_features-Copy1.las


In [25]:
las_file = str(LAS_files[0])

In [106]:
pipe_LASreader =\
{
  "pipeline":[
    {
      "type":"readers.las",
      "filename":las_file,
      "use_eb_vlr": "true"
    }
  ]
}
# """%lasinput
#print(pipe_reader)

pipeline = pdal.Pipeline(json.dumps(pipe_LASreader))
pipeline.validate()
print(pipeline.validate())
%time n_points = pipeline.execute()
lidar_df = pd.DataFrame(pipeline.arrays[0])
print("Number of points in LiDAR:", n_points)
#print(lidar_df)
lidar_df.head()

True
CPU times: user 1.62 s, sys: 92.2 ms, total: 1.71 s
Wall time: 1.7 s
Number of points in LiDAR: 2044148


,X,Y,Z,Intensity,ReturnNumber,NumberOfReturns,ScanDirectionFlag,EdgeOfFlightLine,Classification,ScanAngleRank,...,Coplanar,HeightAboveGround,Linearity,Planarity,Scattering,Omnivariance,Anisotropy,Eigentropy,Eigen_Sum,Curvature_Change
0,535030.20,5278937.90,455.43,0,1,1,0,0,2,0.0,...,1,0.00,0.410420,0.584518,0.005062,0.001637,0.994938,NaN,0.018132,0.003174
1,535026.84,5278936.73,455.77,0,1,1,0,0,1,0.0,...,1,0.51,0.276498,0.703042,0.020460,0.001875,0.979540,61644.562354,0.013318,0.011732
2,535026.40,5278935.53,455.73,0,1,1,0,0,1,0.0,...,0,0.33,0.565298,0.376374,0.058328,0.003101,0.941672,94534.346613,0.015761,0.039067
3,535026.47,5278936.56,455.73,0,1,1,0,0,2,0.0,...,1,0.00,0.625888,0.363328,0.010784,0.001968,0.989216,0.000000,0.017125,0.007787
4,535026.50,5278935.51,455.72,0,1,1,0,0,2,0.0,...,1,0.00,0.236058,0.742906,0.021036,0.001846,0.978964,0.000000,0.013061,0.011785


In [159]:
(lidar_df[["Eigen_Sum", "Eigenvalue0", "Eigenvalue1", "Eigenvalue2", "Curvature_Change", 'Curvature', 'RadialDensity',
       'Coplanar', 'HeightAboveGround', 'Linearity', 'Planarity', 'Scattering',
       'Omnivariance', 'Anisotropy', 'Eigentropy', 'Eigen_Sum',]].head(100))

,Eigen_Sum,Eigenvalue0,Eigenvalue1,Eigenvalue2,Curvature_Change,Curvature,RadialDensity,Coplanar,HeightAboveGround,Linearity,Planarity,Scattering,Omnivariance,Anisotropy,Eigentropy,Eigen_Sum
0,0.018132,0.000058,0.006704,0.011371,0.003174,0.003174,18.083996,1,0.00,0.410420,0.584518,0.005062,0.001637,0.994938,NaN,0.018132
1,0.013318,0.000156,0.005525,0.007637,0.011732,0.011732,34.049239,1,0.51,0.276498,0.703042,0.020460,0.001875,0.979540,61644.562354,0.013318
2,0.015761,0.000616,0.004589,0.010556,0.039067,0.039067,43.419495,0,0.33,0.565298,0.376374,0.058328,0.003101,0.941672,94534.346613,0.015761
3,0.017125,0.000133,0.004626,0.012366,0.007787,0.007787,35.601001,1,0.00,0.625888,0.363328,0.010784,0.001968,0.989216,0.000000,0.017125
4,0.013061,0.000154,0.005590,0.007317,0.011785,0.011785,43.091237,1,0.00,0.236058,0.742906,0.021036,0.001846,0.978964,0.000000,0.013061
5,0.014625,0.000754,0.005692,0.008179,0.051563,0.051563,44.792207,0,0.35,0.303988,0.603806,0.092206,0.003274,0.907794,0.000000,0.014625
6,0.013889,0.000156,0.004391,0.009342,0.011215,0.011215,34.019398,1,0.00,0.529988,0.453338,0.016674,0.001856,0.983326,0.000000,0.013889
7,0.017052,0.000140,0.005454,0.011457,0.008216,0.008216,34.347655,1,0.00,0.523950,0.463823,0.012227,0.002061,0.987773,0.000000,0.017052
8,0.020663,0.000256,0.003757,0.016650,0.012389,0.012389,30.706983,0,0.00,0.774356,0.210269,0.015375,0.002520,0.984625,0.000000,0.020663
9,0.012295,0.000911,0.003076,0.008307,0.074113,0.074113,42.673455,0,0.35,0.629734,0.260582,0.109684,0.002856,0.890316,0.000000,0.012295


In [166]:
np.max(lidar_df[ "Eigenvalue0"])

1.0576163530349731

In [112]:
print(lidar_df[ "Eigenvalue0"] + lidar_df["Eigenvalue1"]  + lidar_df["Eigenvalue2"])

0          0.018132
1          0.013318
2          0.015761
3          0.017125
4          0.013061
5          0.014625
6          0.013889
7          0.017052
8          0.020663
9          0.012295
10         0.013554
11         0.012511
12         0.017954
13         0.012850
14         0.015039
15         0.014793
16         0.014975
17         0.016157
18         0.013298
19         0.020220
20         0.014871
21         0.013338
22         0.021332
23         0.016134
24         0.011641
25         0.013084
26         0.023998
27         0.013791
28         0.015066
29         0.015021
             ...   
2044118    0.036505
2044119    0.027777
2044120    0.111889
2044121    0.024977
2044122    0.026077
2044123    0.031575
2044124    0.027998
2044125    0.028263
2044126    0.032632
2044127    0.025564
2044128    0.042920
2044129    0.056391
2044130    0.089982
2044131    0.032662
2044132    0.031011
2044133    0.086709
2044134    0.033354
2044135    0.069546
2044136    0.038198


In [108]:
len( lidar_df.columns)

39

In [109]:
lidar_df.columns

Index(['X', 'Y', 'Z', 'Intensity', 'ReturnNumber', 'NumberOfReturns',
       'ScanDirectionFlag', 'EdgeOfFlightLine', 'Classification',
       'ScanAngleRank', 'UserData', 'PointSourceId', 'GpsTime', 'Red', 'Green',
       'Blue', 'KDistance', 'LocalReachabilityDistance', 'LocalOutlierFactor',
       'NNDistance', 'Eigenvalue0', 'Eigenvalue1', 'Eigenvalue2', 'Rank',
       'NormalX', 'NormalY', 'NormalZ', 'Curvature', 'RadialDensity',
       'Coplanar', 'HeightAboveGround', 'Linearity', 'Planarity', 'Scattering',
       'Omnivariance', 'Anisotropy', 'Eigentropy', 'Eigen_Sum',
       'Curvature_Change'],
      dtype='object')

In [29]:
 KD-tree

NameError: name 'KD' is not defined

In [100]:
a = np.array([[1,4,2], [3,1,6], [12, 11, 16], [12, 11, 16], [12, 17, 12], [12, 16, 13]])

In [101]:
np.sort(a, axis=1)

array([[ 1,  2,  4],
       [ 1,  3,  6],
       [11, 12, 16],
       [11, 12, 16],
       [12, 12, 17],
       [12, 13, 16]])

In [53]:
print(a.sort(a))

None


In [58]:
np.sum(a, axis=1)

array([ 7, 10, 39])

In [59]:
a[:,1] + a[:,2]

array([ 6,  9, 28])

In [72]:
    evalue1 = a[:,2] # largest
    evalue2 = a[:,1]
    evalue3 = a[:,0] # smallest
    print(evalue1)
    print(evalue2)
    print(evalue3)
    # Linearity
    lambda1 = (evalue1 - evalue2) / evalue1
    print(lambda1)
    # Planarity
    lambda2 = (evalue2 - evalue3) / evalue1
    print(lambda2)

[ 4  6 16]
[ 2  3 12]
[ 1  1 11]
[0.5  0.5  0.25]
[0.25       0.33333333 0.0625    ]


In [78]:
np.divide(np.subtract(evalue1, evalue2), evalue1) 

array([0.5 , 0.5 , 0.25])

In [80]:
lambda4 = pow(evalue1*evalue2*evalue3, 1/3.0)
lambda4

array([ 2.        ,  2.62074139, 12.83013732])

In [89]:
    s = 0
    for elem in a[:,]:
        print(elem)
        s = s + (elem*np.log(elem))
    lambda6 = (-1)*s 

[1 2 4]
[1 3 6]
[11 12 16]


In [85]:
lambda6

array([-26.376848  , -34.50101102, -60.65715382])

In [94]:
a

array([[ 1,  2,  4],
       [ 1,  3,  6],
       [11, 12, 16]])

In [232]:
def replaceZeroes(data):
  min_nonzero = np.min(data[np.nonzero(data)])
  data[data == 0] = min_nonzero
  return data


prob = replaceZeroes(prob)
result = numpy.where(prob > 0.0000000001, numpy.log10(prob), -10)

In [92]:
-np.sum(np.multiply(a, np.log(a)), axis = 1)

array([  -6.93147181,  -14.04639368, -100.55714735])

In [93]:
np.multiply(a, np.log(a))

array([[ 0.        ,  1.38629436,  5.54517744],
       [ 0.        ,  3.29583687, 10.75055682],
       [26.376848  , 29.8188798 , 44.36141956]])

In [103]:
a.T.shape

(3, 6)

In [105]:
a.T

array([[ 1,  3, 12, 12, 12, 12],
       [ 4,  1, 11, 11, 17, 16],
       [ 2,  6, 16, 16, 12, 13]])

In [207]:
np.array(lidar_df[ "Eigenvalue0"])

array([5.75580743e-05, 1.56242924e-04, 6.15718774e-04, ...,
       7.71499649e-02, 3.17722466e-03, 3.17032427e-01])

In [223]:
D = np.array([lidar_df[ "Eigenvalue0"], lidar_df["Eigenvalue1"], lidar_df["Eigenvalue2"]])
D

array([[5.75580743e-05, 1.56242924e-04, 6.15718774e-04, ...,
        7.71499649e-02, 3.17722466e-03, 3.17032427e-01],
       [6.70391694e-03, 5.52505953e-03, 4.58879722e-03, ...,
        7.79708624e-01, 1.98582858e-02, 6.99384034e-01],
       [1.13706617e-02, 7.63655407e-03, 1.05561931e-02, ...,
        1.58507824e+00, 5.44323325e-02, 1.22416902e+00]])

In [225]:
D[:,1]

array([0.00015624, 0.00552506, 0.00763655])

In [216]:
D.shape
D.T
D

array([[5.75580743e-05, 6.70391694e-03, 1.13706617e-02],
       [1.56242924e-04, 5.52505953e-03, 7.63655407e-03],
       [6.15718774e-04, 4.58879722e-03, 1.05561931e-02],
       ...,
       [7.71499649e-02, 7.79708624e-01, 1.58507824e+00],
       [3.17722466e-03, 1.98582858e-02, 5.44323325e-02],
       [3.17032427e-01, 6.99384034e-01, 1.22416902e+00]])

In [224]:
D.sort(axis=0)
D

array([[5.75580743e-05, 1.56242924e-04, 6.15718774e-04, ...,
        7.71499649e-02, 3.17722466e-03, 3.17032427e-01],
       [6.70391694e-03, 5.52505953e-03, 4.58879722e-03, ...,
        7.79708624e-01, 1.98582858e-02, 6.99384034e-01],
       [1.13706617e-02, 7.63655407e-03, 1.05561931e-02, ...,
        1.58507824e+00, 5.44323325e-02, 1.22416902e+00]])

In [226]:
# np.sum(D, axis=1)
print(D)
# Get eigenvectors
# e1 = V[2] # eigenvector in direction of largest variance
# e2 = V[1] # second eigenvector, perpend. to e1
# e3 = V[0]
# Find the eigenvalues
evalue1 = D[:,2] # largest
evalue2 = D[:,1]
evalue3 = D[:,0] # smallest
print(evalue1, "\n\n\n", evalue2, "\n\n\n", evalue3)

[[5.75580743e-05 1.56242924e-04 6.15718774e-04 ... 7.71499649e-02
  3.17722466e-03 3.17032427e-01]
 [6.70391694e-03 5.52505953e-03 4.58879722e-03 ... 7.79708624e-01
  1.98582858e-02 6.99384034e-01]
 [1.13706617e-02 7.63655407e-03 1.05561931e-02 ... 1.58507824e+00
  5.44323325e-02 1.22416902e+00]]
[0.00061572 0.0045888  0.01055619] 


 [0.00015624 0.00552506 0.00763655] 


 [5.75580743e-05 6.70391694e-03 1.13706617e-02]


In [231]:
np.min(D)

0.0

In [227]:
lambda6 = -np.sum(np.multiply(D, np.log(D)), axis = 0)

/opt/anaconda/envs/pyntcloud/lib/python3.7/site-packages/ipykernel_launcher.py:1: RuntimeWarning: divide by zero encountered in log
  """Entry point for launching an IPython kernel.
/opt/anaconda/envs/pyntcloud/lib/python3.7/site-packages/ipykernel_launcher.py:1: RuntimeWarning: invalid value encountered in multiply
  """Entry point for launching an IPython kernel.


In [184]:
evalue1

array([0.01137066, 0.00763655, 0.01055619, ..., 1.58507824, 0.05443233,
       1.22416902])

In [186]:
np.multiply(evalue1[:,].T, np.log(evalue1[:,].T))

array([-0.05090325, -0.03722674, -0.04804168, ...,  0.73014056,
       -0.15844147,  0.24760319])

In [134]:
-np.sum(np.multiply(evalue1, np.log(evalue1)), axis = 0)

/opt/anaconda/envs/pyntcloud/lib/python3.7/site-packages/ipykernel_launcher.py:1: RuntimeWarning: divide by zero encountered in log
  """Entry point for launching an IPython kernel.
/opt/anaconda/envs/pyntcloud/lib/python3.7/site-packages/ipykernel_launcher.py:1: RuntimeWarning: invalid value encountered in multiply
  """Entry point for launching an IPython kernel.


array([ 0.08501871,  0.06731788,  0.07730024, ..., -0.33846325,
        0.25454334,  0.36665665])

In [136]:
(-np.sum(np.multiply(D[:,], np.log(D[:,])), axis = 0)).shape

/opt/anaconda/envs/pyntcloud/lib/python3.7/site-packages/ipykernel_launcher.py:1: RuntimeWarning: divide by zero encountered in log
  """Entry point for launching an IPython kernel.
/opt/anaconda/envs/pyntcloud/lib/python3.7/site-packages/ipykernel_launcher.py:1: RuntimeWarning: invalid value encountered in multiply
  """Entry point for launching an IPython kernel.


(2044148,)

In [137]:
np.multiply(D[:,], np.log(D[:,]))

/opt/anaconda/envs/pyntcloud/lib/python3.7/site-packages/ipykernel_launcher.py:1: RuntimeWarning: divide by zero encountered in log
  """Entry point for launching an IPython kernel.
/opt/anaconda/envs/pyntcloud/lib/python3.7/site-packages/ipykernel_launcher.py:1: RuntimeWarning: invalid value encountered in multiply
  """Entry point for launching an IPython kernel.


array([[-5.61923141e-04, -1.36932839e-03, -4.55183664e-03, ...,
        -1.97658531e-01, -1.82745931e-02, -3.64191386e-01],
       [-3.35535287e-02, -2.87218079e-02, -2.47067144e-02, ...,
        -1.94018786e-01, -7.78272818e-02, -2.50068456e-01],
       [-5.09032546e-02, -3.72267411e-02, -4.80416840e-02, ...,
         7.30140563e-01, -1.58441468e-01,  2.47603191e-01]])

In [172]:
(D * np.log(D)).T

/opt/anaconda/envs/pyntcloud/lib/python3.7/site-packages/ipykernel_launcher.py:1: RuntimeWarning: divide by zero encountered in log
  """Entry point for launching an IPython kernel.
/opt/anaconda/envs/pyntcloud/lib/python3.7/site-packages/ipykernel_launcher.py:1: RuntimeWarning: invalid value encountered in multiply
  """Entry point for launching an IPython kernel.


array([[-5.61923141e-04, -3.35535287e-02, -5.09032546e-02],
       [-1.36932839e-03, -2.87218079e-02, -3.72267411e-02],
       [-4.55183664e-03, -2.47067144e-02, -4.80416840e-02],
       ...,
       [-1.97658531e-01, -1.94018786e-01,  7.30140563e-01],
       [-1.82745931e-02, -7.78272818e-02, -1.58441468e-01],
       [-3.64191386e-01, -2.50068456e-01,  2.47603191e-01]])

In [173]:
np.multiply(D[:,0], np.log(D[:,0]))

array([-0.00056192, -0.03355353, -0.05090325])

In [174]:
np.multiply(D[:,1], np.log(D[:,1]))

array([-0.00136933, -0.02872181, -0.03722674])

In [175]:
np.multiply(D[:,2], np.log(D[:,2]))

array([-0.00455184, -0.02470671, -0.04804168])

In [176]:
lambda7 = np.sum(D, axis=1)
lambda7

array([ 1184.94355398, 12347.36058421, 22213.72501611])

In [154]:
lambda7 = np.sum(D, axis=0)
lambda7

array([0.01813214, 0.01331786, 0.01576071, ..., 2.44193683, 0.07746784,
       2.24058548])